In [9]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census

# Census API Key
from config import census_key


c = Census(census_key, year=2013)

In [10]:
file = 'zipcodes_stl2.csv'
df = pd.read_csv(file)
df_small = df.iloc[0:5,:]
df_small

,ZIP Code,County,Population,Latitude,Longitude
0,63101,Saint Louis City,"2,620",38.631551,-90.19300
1,63102,Saint Louis City,"2,316",38.635200,-90.18702
2,63103,Saint Louis City,"6,900",38.631451,-90.21415
3,63104,Saint Louis City,"18,656",38.610701,-90.21362
4,63105,Saint Louis,"17,667",38.645484,-90.32888


In [11]:
df_census = pd.DataFrame()

for zip in df_small['ZIP Code']:
    census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                                  "B19301_001E",
                                  "B17001_002E"), {'for': 'zip code tabulation area:'+str(zip)})
#     print(census_data)
    df_new = pd.DataFrame(census_data)
    df_census = df_census.append(df_new)
# df_census.head()
# Column Reordering
df_census = df_census.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "NAME": "Name", "zip code tabulation area": "ZIP Code"})

# Add in Poverty Rate (Poverty Count / Population)
df_census["Poverty Rate"] = 100 * \
    df_census["Poverty Count"].astype(
        int) / df_census["Population"].astype(int)

# Final DataFrame
df_census = df_census[["ZIP Code", "Population", "Median Age", "Household Income",
                       "Per Capita Income", "Poverty Count","Poverty Rate"]]

df_census['ZIP Code'] = pd.to_numeric(df_census['ZIP Code'])
df_census.head()

,ZIP Code,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate
0,63101,2613.0,30.4,54417.0,42701.0,610.0,23.344814
0,63102,2216.0,33.6,54018.0,32318.0,142.0,6.407942
0,63103,6824.0,30.4,34719.0,30274.0,1340.0,19.636577
0,63104,19520.0,31.2,45498.0,29780.0,5417.0,27.751025
0,63105,17361.0,33.8,86031.0,57408.0,1206.0,6.946604


In [12]:
#######

df_merge = pd.merge(df_census, df_small, on=('ZIP Code'))
df_merge
######

#New Dependencies

import json
import pprint
# import requests
# import sys
import urllib
from urllib.parse import quote
    
from config import yelp_key
########

In [13]:
API_KEY=yelp_key

API_HOST = 'https://api.yelp.com'
SEARCH_PATH = '/v3/businesses/search'
MATCH_PATH = '/v3/businesses/matches'
SEARCH_PHONE_PATH = '/v3/businesses/search/phone'
BUSINESS_PATH = '/v3/businesses/'  # Business ID will come after slash.

# Defaults for our simple example.
DEFAULT_TERM = 'dinner'
DEFAULT_LOCATION = 'San Francisco, CA'
SEARCH_LIMIT = 10
#########


def request(host, path, api_key, url_params=None):
    url_params = url_params or {}
    url = '{0}{1}'.format(host, quote(path.encode('utf8')))
    headers = {
        'Authorization': 'Bearer %s' % api_key,
    }

    print(u'Querying {0} ...'.format(url))
#     print(headers)

    response = requests.request('GET', url, headers=headers, params=url_params)
#     my_url=url
    return response.json() #, my_url

#######

In [15]:
df_merge.dtypes
df_merge['Latitude'] = df_merge['Latitude'].astype(str)
df_merge['Longitude'] = df_merge['Longitude'].astype(str)
df_merge.dtypes
########

df_businesses = pd.DataFrame()

df_merge['Latitude'] = df_merge['Latitude'].astype(str)
df_merge['Longitude'] = df_merge['Longitude'].astype(str)

for row in df_merge:
#     row_lat = row['Latitude']
#     row_long = row['Longitude']
#     print(row_lat,row_long)
#     url_params={'latitude':row_lat,'longitude':row_long, 'radius':'10000'}
    url_params={'latitude':'38.739650','longitude':'-90.32178', 'radius':'10000'}
    dict_businesses = request(API_HOST, SEARCH_PATH, API_KEY, url_params)

    df_businesses['business_id']=[business["id"] for business in dict_businesses['businesses']]
    df_businesses['name']=[business["name"]for business in dict_businesses['businesses']]
    df_businesses['rating']=[business["rating"]for business in dict_businesses['businesses']]
    df_businesses['zip_code']=[business['location']["zip_code"]for business in dict_businesses['businesses']]
    df_businesses['review_count']=[business["review_count"]for business in dict_businesses['businesses']]
    # try:
    #     df_businesses['price']= [business["price"].map('\$'.format)for business in dict_businesses['businesses']]
    # except:
    #     df_businesses['price']=0
    try:
        if dict_businesses['businesses']['price'] == "$":
            df_businesses['price'] = 1
        elif dict_businesses['business']['price'] == '$$':
            df_businesses['price'] = 2
    except:
        df_businesses['price'] = 0

df_businesses
# pprint.pprint(dict_businesses)

Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...


,business_id,name,rating,zip_code,review_count,price
0,vWjME92WddneF98IqzaNfQ,Sugarfire Smoke House,4.5,63132,1462,0
1,F0AAzMdhv2RyjwHVned5bQ,Hendel's Restaurant,4.5,63031,249,0
2,R8t9g5nvi7VFyS8zsgmj8Q,Salt + Smoke,4.5,63130,1431,0
3,ty6H-2zkK-8IQbhjUfs-8g,Pearl Cafe,4.5,63031,332,0
4,YmUVCO5-ZdSenHVKEJE1rA,Cathy's Kitchen,4.0,63135,132,0
5,X8hwEPRoRcLjbKVlVt1b1g,Saint Louis Zoo,4.5,63110,730,0
6,2DWqlmcMV9CxzUDYhzY04Q,Drake's Place,4.5,63135,77,0
7,pt5huZn_qmSi-4ISWAaYgA,Thai Kitchen - Florissant,5.0,63031,104,0
8,1YflE3DkiCZGgLnf3paLnA,Forest Park,5.0,63112,262,0
9,HKS-63JzfR0DMNVLBuokYw,Mission Taco Joint,4.0,63130,948,0
